<a href="https://colab.research.google.com/github/batuhanyndny/notebooks/blob/master/steganography.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from PIL import Image
import binascii
import optparse

def rgb2hex(r, g, b):
  return '#{:02x}{:02x}{:02x}'.format(r, g, b)

def hex2rgb(hexcode):
  return tuple(map(ord,hexcode[1:].decode('hex')))

def str2bin(msg):
  binary = bin(int(binascii.hexlify(msg), 16))
  return binary

def bin2str(binary):
  msg = binascii.unhexlify('%x' %(int('0b'+binary, 2)))
  return msg

def encode(hexcode, digit):
  if hexcode[-1] in ('0', '1', '2', '3', '4', '5'):
    hexcode = hexcode[:-1] + digit
    return hexcode
  else:
    return None
  
def decode(hexcode):
  if hexcode[-1] in ('0', '1'):
    return hexcode[-1]
  else:
    return None

def hide(fname, msg):
  img = Image.open(fname)
  binary = str2bin(msg) + '1111111111111110'
  if img.mode in ('RGBA'):
    img = img.convert('RGBA')
    datas = img.getdata()
    
    newData = []
    digit = 0
    temp = ''
    for pixel in datas:
      if (digit < len(binary)):
        newpix = encode(rgb2hex(pixel[0], pixel[1], pixel[2]),binary[digit])
        if newpix == None:
          newData.append(pixel)
        else:
          r, g, b = hex2rgb(newpix)
          newData.append((r, g, b, 255))
          digit += 1
      else:
        newData.append(pixel)
        
    img.putdata(newData)
    img.save(fname,'PNG')
    return 'Completed.'
  return 'incorrect image mode, operation failed'




def retr(fname):
  img = Image.open(fname)
  binary = ''
  if img.mode in ('RGBA'):
    img = img.convert('RGBA')
    datas = img.getdata()
    for pixel in datas:
      digit = decode(rgb2hex(pixel[0], pixel[1], pixel[2]))
      if digit == None:
        pass
      else:
        binary = binary + digit
        if binary[-16:] == '1111111111111110':
          print('success')
          return bin2str(binary[:-16])
    return bin2str(binary)
  return 'incorrect image mode, operation failed'

def Main():
  parser = optparse.OptionParser('usage %prog'+\
                                 '-e / -d <target file>')
  parser.add_option('-e', dest='hide', type='string', \
                    help='target picture path to hide text')
  parser.add_option('-d', dest='retr', type='string', \
                    help='target picture path to retrieve text')
  
  (options, args) = parser.parse_args()
  
  if (options.hide) != None:
    text = input('Enter a message to hide: ')
    print(hide(options.hide, text))
  elif options.retr != None:
    print(retr(options.retr))
  else:
    print(parser.usage)
    exit(0)
    
if __name__ == '__main__':
  Main()

Usage: usage ipykernel_launcher.py-e / -d <target file>

ipykernel_launcher.py: error: no such option: -f


SystemExit: ignored

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2890: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
